In [1]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.2/32.2 MB 7.5 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import polars as pl
pl.__version__

'1.7.1'

**DataFrame-Testing**

In [5]:
from datetime import datetime

df = pl.DataFrame(
    {
        "integer": [1, 2, 3],
        "date": [
            datetime(2025, 1, 1),
            datetime(2025, 1, 2),
            datetime(2025, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"],
    }
)


In [4]:
df

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""


In [7]:
!pip install faker

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


**Make Fake Dataset**


Uncomment this if you are running this code first time , to create Dataset.

Read CSV

lzDF = pl.scan_csv("10000000.csv")
lzDF

In [5]:
lzDF.collect_schema()

Schema([('user_id', Int64),
        ('name', String),
        ('date_of_birth', String),
        ('country_of_birth', String),
        ('email', String),
        ('record_year', Int64),
        ('score', Int64)])

**GPU MEMORY-UTILIZATION BEFORE LOADING DATASET ON GPU**

In [6]:
!nvidia-smi

Fri Sep 20 09:34:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   29C    P0              59W / 400W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**As There are 8 GPU'S and Memory Usage is 0 i.e 0MiB/81920MiB**

In [7]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

**RUN ON CPU**

In [8]:
%%time

cpu_query = (
    lzDF
    .group_by("country_of_birth")
    .agg(pl.col("score").mean())
    .sort(by="score", descending=True)
    .head()
    .collect()
)
cpu_query

CPU times: user 19.4 s, sys: 3.66 s, total: 23.1 s
Wall time: 444 ms


country_of_birth,score
str,f64
"""Jamaica""",503.339533
"""Estonia""",503.291502
"""Ghana""",503.221561
"""Barbados""",503.107001
"""Bolivia""",502.871664


**CPU TAKE TOTAL 23.1 s**

Point to note here is that Data is already on CPU memory

In [8]:
# isntalling polar with cuda support
!pip install cudf-polars-cu12

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
!nvidia-smi

Fri Sep 20 09:35:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   29C    P0              59W / 400W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**Still 0mib bcz it was run on CPU**



**NOW RUN ON GPU**

First it will load/transfer data from CPU to GPU


Take more time than CPU

In [10]:
%%time

gpu_query = (
    lzDF
    .group_by("country_of_birth")
    .agg(pl.col("score").mean())
    .sort(by="score", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
gpu_query

CPU times: user 11.6 s, sys: 20.1 s, total: 31.7 s
Wall time: 17.9 s


country_of_birth,score
str,f64
"""Jamaica""",503.339533
"""Estonia""",503.291502
"""Ghana""",503.221561
"""Barbados""",503.107001
"""Bolivia""",502.871664


**As first time it take 31.7s to run and load dataset on GPU**

In [11]:
!nvidia-smi

Fri Sep 20 09:39:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0              65W / 400W |  40751MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**Now look it take Half of GPU Memory**



**Lets again run this query**

In [12]:
%%time

gpu_query = (
    lzDF
    .group_by("country_of_birth")
    .agg(pl.col("score").mean())
    .sort(by="score", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
gpu_query

CPU times: user 131 ms, sys: 113 ms, total: 243 ms
Wall time: 232 ms


country_of_birth,score
str,f64
"""Jamaica""",503.339533
"""Estonia""",503.291502
"""Ghana""",503.221561
"""Barbados""",503.107001
"""Bolivia""",502.871664


**This time it take only 243 ms i.e. 0.243 seconds**

**Fantastic....RIGHT!!**

**Now check GPU Memory again**

In [13]:
!nvidia-smi

Fri Sep 20 09:43:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0              66W / 400W |  40751MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**Not much changes in memory usage**

**Lets Run other query on GPU**

In [15]:
%%time


new_query = (
    lzDF
    .filter((pl.col("record_year") > 2015) & (pl.col("score") > 50))  # Filter rows based on record year and score
    .group_by(["country_of_birth", "record_year"])  # Group by country and record year
    .agg([
        pl.col("score").mean().alias("avg_score"),  # Calculate average score
        pl.col("score").sum().alias("total_score"),  # Calculate total score per group
        pl.col("user_id").count().alias("user_count"),  # Count the number of users per group
    ])
    .sort(by="total_score", descending=True)  # Sort by total score in descending order
    .head(5)  # Select top 5 countries with the highest total score
    .collect(engine=gpu_engine)  # Collect using GPU engine
)

new_query

CPU times: user 152 ms, sys: 126 ms, total: 278 ms
Wall time: 256 ms


country_of_birth,record_year,avg_score,total_score,user_count
str,i64,f64,i64,u32
"""Korea""",2017,545.625523,782427,1434
"""Congo""",2019,528.550409,775912,1468
"""Korea""",2019,523.653324,771865,1474
"""Congo""",2020,530.783439,749997,1413
"""Korea""",2021,521.718794,743971,1426


**This time it take 278ms that is only .278 second**

Polars amazing Speed !!

In [16]:
!nvidia-smi

Fri Sep 20 09:46:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0              66W / 400W |  40751MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**Memory usage remains unchanged, as the required allocation occurs upfront. However, transferring data to the GPU has resulted in a 30X increase in execution time.**

**Now running the same query on CPU**

In [17]:
%%time


new_query = (
    lzDF
    .filter((pl.col("record_year") > 2015) & (pl.col("score") > 50)) 
    .group_by(["country_of_birth", "record_year"]) 
    .agg([
        pl.col("score").mean().alias("avg_score"),  
        pl.col("score").sum().alias("total_score"),  
        pl.col("user_id").count().alias("user_count"),  
    ])
    .sort(by="total_score", descending=True)  
    .head(5)  
    .collect() )

new_query

CPU times: user 15.4 s, sys: 15.1 s, total: 30.5 s
Wall time: 355 ms


country_of_birth,record_year,avg_score,total_score,user_count
str,i64,f64,i64,u32
"""Korea""",2017,545.625523,782427,1434
"""Congo""",2019,528.550409,775912,1468
"""Korea""",2019,523.653324,771865,1474
"""Congo""",2020,530.783439,749997,1413
"""Korea""",2021,521.718794,743971,1426
